In [6]:
# Data loading and preprocessing
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

# Load Oxford Pets dataset
splits = tfds.load('oxford_iiit_pet', as_supervised=True, split=['train[:80%]', 'train[80%:]', 'test'])
(train_dataset, validation_dataset, test_dataset) = splits

# Define preprocessing function
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalize to [0,1]
    return image, label

# Apply preprocessing to datasets
batch_size = 32
train_dataset = train_dataset.map(preprocess_image).shuffle(1000).batch(batch_size)
validation_dataset = validation_dataset.map(preprocess_image).batch(batch_size)
test_dataset = test_dataset.map(preprocess_image).batch(batch_size)

# Create model using transfer learning
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model layers
base_model.trainable = False

# Build the model architecture
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(37, activation='softmax')  # 37 pet categories
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
    ]
)

# Fine-tuning: unfreeze some top layers of the base model
base_model.trainable = True
for layer in base_model.layers[:-10]:  # Freeze all but the last 10 layers
    layer.trainable = False

# Recompile with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Lower learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training with fine-tuning
history_fine = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

# Evaluate on test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {test_accuracy:.4f}")

# Generate predictions for the first batch of test images
images, labels = next(iter(test_dataset))
predictions = model.predict(images)
predicted_classes = np.argmax(predictions, axis=1)

# Display some example predictions
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.title(f"True: {labels[i]}\nPred: {predicted_classes[i]}")
    plt.axis("off")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

KeyboardInterrupt: 

NameError: name 'model' is not defined